# Imports

In [ ]:
import os
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env

# Test environment

In [ ]:
# !curl www.atarimania.com/roms/Roms.rar -o "../input/roms.rar"
# !unrar x "../input/roms.rar" "../input/"
# !unzip "../input/*zip" -d "../input/"
# !cd ../input && rm 'HC ROMS'  'HC ROMS.zip'   roms.rar   ROMS.zip -r

In [ ]:
!python -m atari_py.import_roms ../input/ROMS/

In [ ]:
environment_name = 'Breakout-v0'
env = gym.make(environment_name)

In [ ]:
env.reset()

In [ ]:
env.action_space

In [ ]:
env.observation_space

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

# Vectorise environment and train model

In [ ]:
env = make_atari_env(environment_name, n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

In [ ]:
env.reset()
env.render()
env.close()

In [ ]:
log_path = os.path.join('../output','training', 'logs')

In [ ]:
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=200_000)

# Save and reload model

In [ ]:
a2c_path = os.path.join('../output', 'training', 'saved models', 'a2c_breakout')

In [ ]:
model.save(a2c_path)
del model

In [ ]:
model = A2C.load(a2c_path, env)

# Evaluate and test

In [ ]:
env = make_atari_env(environment_name, n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir={log_path}